In [1]:
import numpy as np
import tensorflow as tf
import load_samples as ls
import math


#number of dimensions used and steps when unrolling
input_dim = 1
input_steps = 5

hidden_layer = 5

labels = ['curve_m','straight','wall','object']
classes = len(labels)

#percentage used for test data
test_rate = 0.1

In [11]:
data_train_x = []
data_train_y = []
data_test_x = []
data_test_y = []
i = 0

for label in labels:
    samples = ls.load_sample(labels = label)
    samples = ls.convert_to_input(samples)
    
    low = int(test_rate*len(samples))
    data_train = samples[low:]
    data_test = samples[:low]
    
    data_train_x.extend(data_train)
    data_train_y.extend([i]*len(data_train))
    data_test_x.extend(data_test)
    data_test_y.extend([i]*len(data_test))
    
    i += 1
    

todo
todo
todo
todo
todo
todo
todo
todo
todo
todo


In [3]:
data_train_x = [[[0],[1],[1],[1],[1]], [[0],[0],[1],[1],[1]], 
                [[0],[0],[0],[1],[1]], [[0],[0],[0],[0],[1]], 
                [[0],[0],[0],[0],[0]], [[1],[1],[1],[1],[1]], 
                [[1],[0],[1],[0],[1]], [[1],[1],[0],[0],[0]], 
                [[1],[0],[0],[0],[0]], [[1],[1],[0],[0],[1]], 
                [[1],[1],[1],[0],[1]], [[1],[1],[1],[1],[0]], 
                [[1],[1],[1],[0],[0]], [[1],[1],[0],[1],[0]], 
                [[1],[0],[0],[1],[1]], [[1],[0],[0],[0],[1]]]
data_test_x = data_train_x
data_train_y = [1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0]
data_test_y = data_train_y

In [4]:
def convert_to_onehot(labels):
    new_labels = []
    for y in labels:
        label = [0]*classes
        label[y] = 1
        new_labels.append(label)
    return new_labels

xtrain = np.array(data_train_x)
xtest = np.array(data_test_x)

ytrain = convert_to_onehot(data_train_y)
ytest = convert_to_onehot(data_test_y)
ytrain = np.array(ytrain)
ytest = np.array(ytest)

In [5]:
tf.logging.set_verbosity(tf.logging.ERROR)
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, input_steps, input_dim])
y = tf.placeholder(tf.float32, [None, classes])

W = tf.Variable(tf.random_normal([hidden_layer, classes]))
b = tf.Variable(tf.zeros([classes]))

#x = [tf.placeholder(tf.float32, [None, input_dim]) for _ in range(input_steps)]
#y = tf.placeholder(tf.float32, [None, classes])
x2 = tf.unstack(x, input_steps, 1)

rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer)
outputs, state = tf.contrib.rnn.static_rnn(rnn_cell, x2, dtype=tf.float32)
rnn = tf.matmul(outputs[-1],W) + b
rnn_pred = tf.argmax(rnn,1)

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=rnn))

optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), rnn_pred)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
steps = 10000

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(steps):
    sess.run(optimizer, feed_dict={x: xtrain, y:ytrain})

In [7]:
sess.run(accuracy, feed_dict={x: xtest, y: ytest})

1.0

In [8]:
def feed_data(data):
    if len(data) < input_dim: raise ValueError('Data too short')
    results = []
    for i in range(len(data) - input_steps + 1):
        batch = np.array(data[:input_steps])
        res = sess.run(rnn_pred, feed_dict={x:[batch]})
        results.append([list(res), list(batch)])
        data.pop(0)
    return results

In [9]:
feed_data([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[0],[0],[1],[1],[1],[1],[1]])

[[[1], [array([1]), array([1]), array([1]), array([1]), array([1])]],
 [[1], [array([1]), array([1]), array([1]), array([1]), array([1])]],
 [[1], [array([1]), array([1]), array([1]), array([1]), array([1])]],
 [[1], [array([1]), array([1]), array([1]), array([1]), array([1])]],
 [[1], [array([1]), array([1]), array([1]), array([1]), array([1])]],
 [[1], [array([1]), array([1]), array([1]), array([1]), array([1])]],
 [[1], [array([1]), array([1]), array([1]), array([1]), array([1])]],
 [[1], [array([1]), array([1]), array([1]), array([1]), array([1])]],
 [[1], [array([1]), array([1]), array([1]), array([1]), array([1])]],
 [[0], [array([1]), array([1]), array([1]), array([1]), array([0])]],
 [[0], [array([1]), array([1]), array([1]), array([0]), array([0])]],
 [[0], [array([1]), array([1]), array([0]), array([0]), array([1])]],
 [[0], [array([1]), array([0]), array([0]), array([1]), array([1])]],
 [[1], [array([0]), array([0]), array([1]), array([1]), array([1])]],
 [[1], [array([0]), 